# BigQuery data query

In [1]:
from google.cloud import bigquery
client = bigquery.Client()

In [2]:
sql = """
SELECT
  name,
  gender,
  COUNT(name) AS num_names
FROM
  `bigquery-public-data.usa_names.usa_1910_current`
GROUP BY
  name,
  gender
"""
names_df = client.query(sql).to_dataframe()
print(names_df.shape)
names_df.head()

(35236, 3)


,name,gender,num_names
0,Mary,F,5597
1,Annie,F,3994
2,Anna,F,5566
3,Margaret,F,5509
4,Helen,F,4879


In [3]:
# write to csv
names_df[['name', 'gender']].to_csv("../data/us-names.csv", index=False)

# Writing to sharded tfrecords

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:
def write_tfrecord(prefix, chunk, idx):
    # Set writing options with compression
    #options = tf.io.TFRecordOptions(compression_type="ZLIB", compression_level=9)
    options = None
    
    with tf.io.TFRecordWriter("../data/tfrecords/us_names_"+prefix+"_{:>03d}".format(idx)+".tfrecord", options=options) as writer:       
        for row in chunk.values:
            features, label = row[:-1], row[-1]
            
            name = tf.train.Feature(bytes_list=tf.train.BytesList(value=[features[0].encode("utf-8")]))
            gender = tf.train.Feature(bytes_list=tf.train.BytesList(value=[label.encode("utf-8")]))

            data_dict = tf.train.Features(feature={"name": name, "gender": gender})
            example = tf.train.Example(features=data_dict)

            writer.write(example.SerializeToString())

def serialize(chunk_df):
    # Serializes inputs from a pandas dataset (read in chunks)   
    # Write each chunk into individual tfrecord (sharding)
    for idx, chunk in enumerate(chunk_df):
        chunk.dropna(how="any", inplace=True)
        train_chunk, val_chunk, test_chunk = np.split(chunk.sample(frac=1), [int(.7*len(chunk)), int(.9*len(chunk))])

        write_tfrecord("train", train_chunk, idx)
        write_tfrecord("val", val_chunk, idx)
        write_tfrecord("test", test_chunk, idx)

In [7]:
%%time
# Each chunk has 20K names
names_df = pd.read_csv("../data/us-names.csv", chunksize=20000, engine='c', iterator=True)
serialize(names_df)

CPU times: user 1.11 s, sys: 3.33 ms, total: 1.11 s
Wall time: 1.16 s


In [8]:
! gsutil cp ../data/tfrecords/*.tfrecord gs://leo-us-name-gender-us-central1/data/

Copying file://../data/tfrecords/us_names_test_000.tfrecord [Content-Type=application/octet-stream]...
Copying file://../data/tfrecords/us_names_test_001.tfrecord [Content-Type=application/octet-stream]...
Copying file://../data/tfrecords/us_names_train_000.tfrecord [Content-Type=application/octet-stream]...
Copying file://../data/tfrecords/us_names_train_001.tfrecord [Content-Type=application/octet-stream]...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://../data/tfrecords/us_names_val_000.tfrecord [Content-Type=application/octet-stream]...
Copying file://../data/tfrecords/us_names_val_001.tfrecord [Content-Type=application/octet-stream]...

Operation completed over 6 objects/1.9 MiB.                                      
